# For testing code, nothing more


In [7]:
import os

# Doc for download https://idr.openmicroscopy.org/about/download.html

S_PATH = "/".join(os.path.realpath("test").split(os.sep)[:-1])

class data_manager:

    def __init__(self, data_dir = None) -> None:
        if data_dir is None:
            self.data_dir = os.path.join(S_PATH, "../data")
            print(f"Data directory is not specified, using default: {self.data_dir}")
        
        # Save file empty file to data directory now
        # TODO: Check if file exists
        #self.data_file = os.path.join(self.data_dir, "test.csv")
        #with open(self.data_file, "w") as f:
        #    f.write("")

    


        pass

    def download_data(self, dataset_name):
        if dataset_name == "METABRIC":
            # Get the command
            command = self.idr_download_command("idr0076")

            # Execute the command
            os.system(command)
        pass

    def preprocess(self, dataset_name):
        pass

    def save_data(self, dataset_name):
        pass

    def load_data(self, dataset_name):
        pass

    def idr_download_command(self, idr_id):
        save_dir = os.path.join(self.data_dir, idr_id)
        return f"ascp -TQ -l40m -P 33001 -i 'config/asperaweb_id_dsa.openssh' {idr_id}@fasp.ebi.ac.uk:. {save_dir}"



In [9]:
d= data_manager()

d.download_data("METABRIC")

Data directory is not specified, using default: /Users/dgermen/Documents/GCN Clinical Clinical Outcome Prediction/GNNClinicalOutcomePrediction/bin/../data


sh: ascp: command not found


# Data merging for METABRIC

In [2]:
import pandas as pd
import time

# Read the first file
clinical_data = pd.read_csv('../data/METABRIC/brca_metabric_clinical_data.tsv', sep='\t')

# Read the second file
single_cell_data = pd.read_csv('../data/METABRIC/single_cell_data.csv')

# Record the time before the merge
start_time = time.time()

# Columns representing mean ion counts
mean_ion_count_columns = [
    'HH3_total', 'CK19', 'CK8_18', 'Twist', 'CD68', 'CK14', 'SMA', 'Vimentin',
    'c_Myc', 'HER2', 'CD3', 'HH3_ph', 'Erk1_2', 'Slug', 'ER', 'PR', 'p53', 'CD44',
    'EpCAM', 'CD45', 'GATA3', 'CD20', 'Beta_catenin', 'CAIX', 'E_cadherin', 'Ki67',
    'EGFR', 'pS6', 'Sox9', 'vWF_CD31', 'pmTOR', 'CK7', 'panCK', 'c_PARP_c_Casp3',
    'DNA1', 'DNA2', 'H3K27me3', 'CK5', 'Fibronectin'
]

# Dictionary to keep track of renamed columns
renamed_columns_mapping = {}

# Rename the mean ion count columns
counter = 1 
for col in mean_ion_count_columns:
    new_name = f'Intensity_MeanIntensity_FullStack_c{counter}'
    renamed_columns_mapping[col] = new_name
    single_cell_data.rename(columns={col: new_name}, inplace=True)
    counter += 1

# Merge the two dataframes on 'Sample ID' and 'metabricId'
merged_data = single_cell_data.merge(clinical_data, left_on='metabricId', right_on='Sample ID', how='inner')

# Rename the other columns
columns_to_rename = {
    'Sample ID': 'PID',
    'Age at Diagnosis': 'age',
    'Neoplasm Histologic Grade': 'grade',
    'Tumor Size': 'tumor_size',
    'Chemotherapy': 'treatment',
    'Cancer Type Detailed': 'clinical_type',
    'Overall Survival (Months)': 'OSmonth',
    'Nottingham prognostic index': 'DiseaseStage',
    'Patient\'s Vital Status': 'diseasestatus',
    'Relapse Free Status (Months)': 'DFSmonth',
    'Cancer Type': 'description'
}

merged_data.rename(columns=columns_to_rename, inplace=True)

# Select the columns you want to keep in the final file
desired_columns = [
    'ImageNumber', 'ObjectNumber', 'Location_Center_X', 'Location_Center_Y'
] + [f'Intensity_MeanIntensity_FullStack_c{i}' for i in range(1, len(mean_ion_count_columns))] + [
    'PID', 'grade', 'tumor_size', 'age', 'treatment', 'DiseaseStage',
    'diseasestatus', 'clinical_type', 'DFSmonth', 'OSmonth'
]

final_data = merged_data[desired_columns]

# Record the time after the merge
end_time = time.time()

# Calculate the time taken for the merge
time_taken = end_time - start_time

# Save the mapping to a file for future reference
with open('../data/METABRIC/merged_data_mapping.txt', 'w') as f:
    for key, value in renamed_columns_mapping.items():
        f.write(f"{key} -> {value}\n")

# Save the merged dataframe to a new CSV file
final_data.to_csv('../data/METABRIC/merged_data.csv', index=False)

print(f"The merging process took {time_taken} seconds")


The merging process took 1.2340710163116455 seconds


In [3]:
# View head of merged data
final_data.head()

,ImageNumber,ObjectNumber,Location_Center_X,Location_Center_Y,Intensity_MeanIntensity_FullStack_c1,Intensity_MeanIntensity_FullStack_c2,Intensity_MeanIntensity_FullStack_c3,Intensity_MeanIntensity_FullStack_c4,Intensity_MeanIntensity_FullStack_c5,Intensity_MeanIntensity_FullStack_c6,...,PID,grade,tumor_size,age,treatment,DiseaseStage,diseasestatus,clinical_type,DFSmonth,OSmonth
0,527,1,161.83333,6.000000,1.058000,0.000000,0.000000,0.000000,0.618000,0.000000,...,MB-0000,3.0,22.0,75.65,NO,6.044,Living,Breast Invasive Ductal Carcinoma,138.65,140.5
1,527,2,177.30435,15.539130,0.889243,0.166252,0.166487,0.109148,0.071443,0.026087,...,MB-0000,3.0,22.0,75.65,NO,6.044,Living,Breast Invasive Ductal Carcinoma,138.65,140.5
2,527,3,293.51898,19.886076,0.861557,0.150873,0.099861,0.064506,0.025316,0.063747,...,MB-0000,3.0,22.0,75.65,NO,6.044,Living,Breast Invasive Ductal Carcinoma,138.65,140.5
3,527,4,165.04347,22.210144,1.746239,0.069159,0.043478,0.021739,0.623623,0.014493,...,MB-0000,3.0,22.0,75.65,NO,6.044,Living,Breast Invasive Ductal Carcinoma,138.65,140.5
4,527,5,108.88095,28.523809,0.979274,0.144429,0.125500,0.067714,0.028940,0.057381,...,MB-0000,3.0,22.0,75.65,NO,6.044,Living,Breast Invasive Ductal Carcinoma,138.65,140.5


In [6]:
# Get average for DFSmonth and OSmonth
print(f"Average DFSmonth: {final_data['DFSmonth'].mean()}")
print(f"Average OSmonth: {final_data['OSmonth'].mean()}")

# Count disease status
final_data['diseasestatus'].value_counts()

# Group by PID and print DFSmonth and OSmonth averages and count disease status
grouped_data = final_data.groupby('PID')
print(f"Average DFSmonth: {grouped_data['DFSmonth'].mean().mean()}")
print(f"Average OSmonth: {grouped_data['OSmonth'].mean().mean()}")
print(f"Count disease status: {grouped_data['diseasestatus'].value_counts()}")


Average DFSmonth: 86.03868933280224
Average OSmonth: 94.57488861993205
Average DFSmonth: 87.37644444444445
Average OSmonth: 95.43036303746783
Count disease status: PID      diseasestatus       
MB-0000  Living                   256
MB-0002  Living                  1111
MB-0005  Died of Disease         1088
MB-0010  Died of Disease         1077
MB-0014  Living                  1569
                                 ... 
MB-0897  Died of Other Causes     927
MB-0899  Living                  1096
MB-0901  Living                  1625
MB-0904  Died of Other Causes     786
MB-0906  Died of Other Causes     750
Name: diseasestatus, Length: 404, dtype: int64


In [7]:
# Print all categorical columns' possible values
for col in final_data.columns:
    if final_data[col].dtype == 'object':
        print(f"{col}: {final_data[col].unique()}")

PID: ['MB-0000' 'MB-0002' 'MB-0005' 'MB-0010' 'MB-0014' 'MB-0020' 'MB-0022'
 'MB-0028' 'MB-0035' 'MB-0045' 'MB-0050' 'MB-0060' 'MB-0064' 'MB-0081'
 'MB-0095' 'MB-0099' 'MB-0107' 'MB-0109' 'MB-0111' 'MB-0112' 'MB-0114'
 'MB-0115' 'MB-0116' 'MB-0117' 'MB-0119' 'MB-0120' 'MB-0121' 'MB-0122'
 'MB-0123' 'MB-0124' 'MB-0126' 'MB-0127' 'MB-0128' 'MB-0130' 'MB-0131'
 'MB-0133' 'MB-0134' 'MB-0135' 'MB-0136' 'MB-0138' 'MB-0139' 'MB-0140'
 'MB-0142' 'MB-0143' 'MB-0144' 'MB-0145' 'MB-0150' 'MB-0151' 'MB-0152'
 'MB-0153' 'MB-0154' 'MB-0158' 'MB-0162' 'MB-0163' 'MB-0164' 'MB-0165'
 'MB-0166' 'MB-0167' 'MB-0168' 'MB-0169' 'MB-0170' 'MB-0173' 'MB-0174'
 'MB-0175' 'MB-0176' 'MB-0177' 'MB-0178' 'MB-0179' 'MB-0180' 'MB-0181'
 'MB-0184' 'MB-0185' 'MB-0188' 'MB-0189' 'MB-0191' 'MB-0192' 'MB-0193'
 'MB-0194' 'MB-0195' 'MB-0197' 'MB-0198' 'MB-0199' 'MB-0200' 'MB-0201'
 'MB-0202' 'MB-0203' 'MB-0204' 'MB-0205' 'MB-0206' 'MB-0214' 'MB-0218'
 'MB-0220' 'MB-0221' 'MB-0222' 'MB-0223' 'MB-0224' 'MB-0225' 'MB-0227'
 